# TensorFlow

[How to setup tensorflow in Linux?](https://www.youtube.com/watch?v=6-eEpq7ChKg])

In [6]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.15.0


In [4]:
# Define the computational graph
graph = tf.Graph()
with graph.as_default():
    x = tf.constant(5, name="X_Value")
    y = tf.constant(2, name="Y_value")
    sum = tf.add(x, y, name="xy_sum")

# Execute the graph
with tf.compat.v1.Session(graph=graph) as session:
    result = session.run(sum)
    print("Result:", result)

TensorFlow version: 2.15.0
Result: 7


## Load a dataset

Load and prepare the MNIST dataset. The pixel values of the images range from 0 through 255. Scale these values to a range of 0 to 1 by dividing the values by 255.0. This also converts the sample data from integers to floating-point numbers:

In [8]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 [==============================] - 1s 0us/step


### Build a ML model

In [9]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

[`Sequential`](https://www.tensorflow.org/guide/keras/sequential_model) is useful for stacking layers where each layer has one input [tensor](https://www.tensorflow.org/guide/tensor) and one output tensor. Layers are functions with a known mathematical structure that can be reused and have trainable variables. Most TensorFlow models are composed of layers. This model uses the [`Flatten`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten), [`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense), and [`Dropout`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) layers.

For each example, the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [10]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.11964792, -0.6828836 ,  0.0058978 ,  0.13633962,  1.0843248 ,
        -0.51743245, -0.45635572,  0.1578706 ,  0.34093377, -0.1590127 ]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to *probabilities* for each class

In [11]:
tf.nn.softmax(predictions).numpy()

array([[0.0794889 , 0.04525811, 0.09012191, 0.10267874, 0.26496303,
        0.05340119, 0.05676442, 0.10491348, 0.12598959, 0.07642063]],
      dtype=float32)

Note: It is possible to bake the `tf.nn.softmax` function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output. 

Define a loss function for training using `losses.SparseCategoricalCrossentropy`:

In [12]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)